In [ ]:
%matplotlib inline
%load_ext rpy2.ipython
import pandas as pd
import ggplot

In [ ]:
def summarize(d):
    
    def split_name(d):
        d = d.copy()
        d1 = d.name.str.extract(r'/(\d.+)\.(\d+)', expand=True)
        d2 = d1[0].str.split('_', expand=True)
        d['uid'] = d1[1]
        cols = ('universal_coverage', 'universal_efficacy',
                 'seasonal_coverage', 'seasonal_efficacy')
        for i,c in zip(range(4), cols):
            d[c] = d2[i].astype(float)
        del d['name']
        return d    
    
    def summarize_by_age(d):
        return pd.Series({
            'individuals': d.N_p.mean(),
            'vaccines': d.V_i.sum(),
            'infections': d.IS_i.sum()
        })

    d1 = split_name(d).groupby(
        ['uid','age','universal_coverage','universal_efficacy',
         'seasonal_coverage','seasonal_efficacy']).apply(summarize_by_age).reset_index()
    
    def summarize_by_uid(d):
        return pd.Series({
            'individuals': d.individuals.sum(),
            'vaccines': d.vaccines.sum(),
            'infections': d.infections.sum()
        })
    
    d2 = d1.groupby(['uid','universal_coverage','universal_efficacy',
         'seasonal_coverage','seasonal_efficacy']).apply(summarize_by_uid).reset_index()
    
    return d2

In [ ]:
d = summarize(pd.read_csv('poe_output.csv'))

In [ ]:
d.head()

In [ ]:
%%R -i d -w 1700 -h 700 -u px
library(pacman)
p_load(data.table,ggplot2,magrittr)


d = data.table(d)
d = d[seasonal_efficacy>0.2,lapply(.SD, mean),by=c('universal_coverage','universal_efficacy','seasonal_coverage'),.SDcols=c('individuals','infections','vaccines')]

years = 3
universal_cost_multiplier = 2
population_size = d$individuals %>% mean
baseline_infections = d[universal_coverage==0 & seasonal_coverage==0.5]$infections %>% mean
baseline_cost = (population_size * 0.5 * years)

d$cost = (population_size * d$seasonal_coverage * years) + (population_size * d$universal_coverage * universal_cost_multiplier)
d$cost_effectiveness_ratio = (baseline_cost - d$cost) / (baseline_infections - d$infections)

#print(d %>% head)

ggplot(d, aes(universal_coverage, seasonal_coverage)) +
    geom_tile(aes(fill=cost_effectiveness_ratio), color='white') +
    scale_fill_gradient(low='blue',high='red') +
    facet_grid(. ~ universal_efficacy, labeller = label_both) 